Gleened from https://medium.com/@galen.ballew/transferlearning-b65772083b47



In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

#import inception with pre-trained weights. do not include fully #connected layers
VGG16_base = VGG16(weights='imagenet', include_top=False)

In [ ]:
# add a global spatial average pooling layer
x = VGG16_base.output
x = GlobalAveragePooling2D()(x)

numberOfClasses =10
# add a fully-connected layer
x = Dense(512, activation='relu')(x)

# and a fully connected output/classification layer
predictions = Dense(numberOfClasses, activation='softmax')(x)

# create the full network so we can train on it
VGG16_transfer = Model(input=VGG16_base.input, output=predictions)

for layer in VGG16_base.layers:
    layer.trainable = False

In [ ]:
VGG16_transfer.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
import PIL 
import os
from PIL import Image
# Generate dummy data
import numpy as np
import random
#data = np.random.random((1000, 155,200,3))
#labels = np.random.randint(2, size=(1000, numberOfClasses))

#VGG16_transfer.fit(data,labels, epochs=4, batch_size=16)
    
## !! SET THESE PARAMETERS !! ##
NumIm = 10  # The number of images you want
Its = 0
numD = np.round(NumIm/2)  # Number of daisies


data = np.zeros([NumIm,263,320,3])  # Initialize an array for all the data

# Grab some daisies
for file in os.listdir('./daisy'):
    if (Its > numD-1):
        break
    
    filename = os.fsdecode(file) # Get the file name 
    Im = (Image.open('./daisy/' + filename)) # Load the image
    Im = Im.resize((320, 263), Image.ANTIALIAS)  # Reshape the image to the size we want
    Im = np.asarray(Im)  # Convert to an array
    
    data[Its,:,:,:] = Im # Store the image
    Its += 1
    

# Grab some daisies    
for file in os.listdir('./roses'):
    if (Its > NumIm-1):
        break
    
    filename = os.fsdecode(file) # Get the file name 
    Im = (Image.open('./roses/' + filename)) # Load the image
    Im = Im.resize((320, 263), Image.ANTIALIAS)  # Reshape the image to the size we want
    Im = np.asarray(Im)  # Convert to an array
    
    data[Its,:,:,:] = Im  # Store the image
    Its += 1

# Create some shuffled idices    
newData = np.zeros([NumIm,263,320,3])
newIx = []
for i in range(NumIm):
    newIx.append(i)
    
random.shuffle(newIx)

# Shuffle the images
for i in range(NumIm):
    Indx = newIx[i]
    newData[i,:,:,:] = data[Indx,:,:,:]

data = newData  # New Shuffled data

In [ ]:
x = VGG16_base.output
x = GlobalAveragePooling2D()(x)

# create the full network so we can train on it
VGG16_Bottle = Model(input=VGG16_base.input, output=x)
globAvgBottleOut= VGG16_Bottle.predict(data)
rawBottleOut= VGG16_base.predict(data)

In [ ]:
globAvgBottleOut.shape

In [ ]:
rawBottleOut.shape